# AI vs Original Image Classification Project

In [ ]:
# Step 1: Import required libraries
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
# Step 2: Define paths
base_path = "dataset"  # path where ai_generated and original folders are
categories = ["ai_generated", "original"]


In [ ]:
# Step 3: Create CSV with image paths and labels
data = []
for category in categories:
    folder_path = os.path.join(base_path, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        data.append([img_path, category])

df = pd.DataFrame(data, columns=["image_path", "label"])
df.to_csv("image_labels.csv", index=False)
print("CSV created with image paths and labels.")


In [ ]:
# Step 4: Load data and preprocess
IMG_SIZE = 128
images = []
labels = []

for index, row in df.iterrows():
    img = cv2.imread(row["image_path"])
    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(row["label"])

X = np.array(images) / 255.0
le = LabelEncoder()
y = le.fit_transform(labels)
y = to_categorical(y)


In [ ]:
# Step 5: Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Step 6: Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Step 7: Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# Step 8: Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.2f}")


In [ ]:
# Step 9: Save the model
model.save("ai_vs_original_classifier.h5")
